In [3]:
# Lets build a regression model to predict sepal length from sepal width, using iris.csv

import pandas as pd

# Load the data
iris = pd.read_csv('iris.csv')

# Split the data into X and y [Univariate regression]
X = iris[['sepal_width']]
y = iris['sepal_length']

# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Create a linear regression model
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

model

/Users/pgmenon/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


LinearRegression()

In [5]:
# get size of X_train
print(len(X_train))

print(len(X_test))

112
38


In [5]:
# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)
print('R^2:', r2_score(y_test, y_pred))

Mean Squared Error: 0.655536071343486
R^2: 0.00017524291267712044


In [7]:
# Visualize the relationship between sepal width and sepal length with the regression line in red , using plotly    
import plotly.express as px

fig = px.scatter(iris, x='sepal_width', y='sepal_length', title='Sepal Width vs Sepal Length')
fig.add_traces(px.line(x=X_test['sepal_width'], y=y_pred).data)

fig.show()

# Lets build various models using h2o 

In [6]:
import h2o
h2o.init(max_mem_size='2G')

Checking whether there is an H2O instance running at http://localhost:54321 .....